In [ ]:
import pandas as pd
import numpy as np
import os

# --------------------------
# 配置参数（核心优化）
# --------------------------
CONFIG = {
    "factortable_path": r'./factortable.parquet',
    "selection_result_path": r'./short_term_selection_optimized.csv',
    "daily_result_dir": r'./daily_short_term_optimized',
    "log_path": r'./short_term_selection_optimized_log.txt',
    "top_n": 150,  # 目标100-200，取中间值
    # 优化：强化形态筛选（排除伪加速）
    "short_term_shape": {
        "consecutive_up_days_min": 3,        # 连续上涨≥3天（不变，保证覆盖）
        "consecutive_up_days_max": 7,         # 缩短至≤7天（避免过度上涨）
        "rise_ratio_30d_min": 0.08,          # 30日涨幅≥8%（不变）
        "rise_ratio_30d_max": 0.4,           # 收紧至≤40%（降低高涨幅风险）
        "daily_rise_min": -0.005,             # 单日涨幅≥0.5%（温和上涨）
        "daily_rise_max": 0.035,              # 单日涨幅≤5%（避免暴涨）
        "volume_ratio_min": 0.9,             # 量能比≥0.9（适度收紧）
        "volume_ratio_consecutive_min": 2,   # 连续≥3天量能比≥1.0
        "rsi_safe_max": 65,                  # 不变
        "ma_trend": True,                    # 不变
        # 优化：竞价筛选（收紧区间）
        "high_open_min": 0.005,               # 高开≥1%（更稳健）
        "high_open_max": 0.020,               # 高开≤2%（规避回落）
        "auction_volume_ratio_min": 0.03     # 竞价量比≥8%（真实资金）
    },
    # 优化：排序权重（突出量价配合）
    "sort_weights": {
        "auction_score": 0.35,        # 竞价得分（提升）
        "price_volume_score": 0.3,    # 量价配合（新增，最高）
        "trend_strength_score": 0.2,  # 趋势强度
        "risk_filter_score": 0.15     # 风险过滤
    }
}

# --------------------------
# 工具函数（不变）
# --------------------------
def init_environment():
    os.makedirs(CONFIG["daily_result_dir"], exist_ok=True)
    with open(CONFIG["log_path"], 'w', encoding='utf-8') as f:
        f.write(f"【短线形态优化版选股启动】{pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    log_msg(f"✅ 环境初始化完成，每日结果目录：{CONFIG['daily_result_dir']}")

def log_msg(msg):
    timestamp = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
    log_line = f"[{timestamp}] {msg}"
    print(log_line)
    with open(CONFIG["log_path"], 'a', encoding='utf-8') as f:
        f.write(log_line + "\n")

def validate_data(df):
    required_cols = [
        'stock_code', 'date', 'close', 'open', 'volume', 'high', 'low',
        'consecutive_up_days', 'is_high_open', 'ma5', 'ma20',
        'macd_line', 'signal_line', 'macd_hist', 'volume_ratio_5d',
        'high_30d', 'pullback_ratio', 'pullback_days', 'bollinger_lower', 'rsi14',
        'auction_rise_ratio', 'auction_volume_ratio'
    ]
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Factortable缺少字段：{missing_cols}")
    log_msg(f"✅ 字段验证通过，共{len(df)}条记录")
    return df

# --------------------------
# 步骤1：计算衍生指标（新增量价配合/每日涨幅）
# --------------------------
def pre_calculate_indicators(df):
    log_msg("开始计算优化版衍生指标...")
    
    # 1. 30日涨幅（不变）
    def calc_30d_rise(group):
        group['rise_ratio_30d'] = (group['close'] - group['close'].shift(30)) / \
                                 group['close'].shift(30).replace(0, 0.0001)
        return group['rise_ratio_30d'].clip(0, 1.0).fillna(0)
    df['rise_ratio_30d'] = df.groupby('stock_code', group_keys=False).apply(calc_30d_rise)
    
    # 2. 每日涨幅（新增：用于筛选温和上涨）
    df['daily_rise_ratio'] = (df['close'] - df['open']) / df['open'].replace(0, 0.0001)
    
    # 3. 连续量能达标记（新增：量价配合验证）
    def calc_consecutive_volume(group):
        vol_over = (group['volume_ratio_5d'] >= 1.0).astype(int)
        consecutive_vol = vol_over.groupby(vol_over.ne(vol_over.shift()).cumsum()).cumsum()
        group['consecutive_volume_days'] = consecutive_vol
        return group
    df = df.groupby('stock_code', group_keys=False).apply(calc_consecutive_volume)
    
    # 4. 价格在5日线上方标记（新增：趋势稳健性）
    df['price_above_ma5'] = (df['close'] >= df['ma5']).astype(int)
    
    log_msg("✅ 优化版衍生指标计算完成")
    return df

# --------------------------
# 步骤2：形态筛选（强化趋势有效性）
# --------------------------
def filter_single_shape(daily_df):
    params = CONFIG["short_term_shape"]
    daily_df = daily_df.copy()
    
    # 核心优化：新增趋势有效性条件
    # 条件1：连续上涨3-7天（收紧上限）
    cond1 = daily_df['consecutive_up_days'].between(
        params['consecutive_up_days_min'], 
        params['consecutive_up_days_max']
    )
    # 条件2：30日涨幅8%-40%（收紧上限）
    cond2 = daily_df['rise_ratio_30d'].between(
        params['rise_ratio_30d_min'], 
        params['rise_ratio_30d_max']
    )
    # 条件3：单日涨幅0.5%-5%（温和加速）
    cond3 = daily_df['daily_rise_ratio'].between(
        params['daily_rise_min'], 
        params['daily_rise_max']
    )
    # 条件4：连续≥3天量能比≥1.0（量价配合）
    cond4 = daily_df['consecutive_volume_days'] >= params['volume_ratio_consecutive_min']
    # 条件5：价格在5日线上方（趋势稳健）
    cond5 = daily_df['price_above_ma5'] == 1
    # 条件6：量能比≥0.9（适度收紧）
    cond6 = daily_df['volume_ratio_5d'] >= params['volume_ratio_min']
    # 条件7：RSI≤65（不变）
    cond7 = daily_df['rsi14'] <= params['rsi_safe_max']
    # 条件8：均线多头（不变）
    cond8 = (daily_df['ma5'] >= daily_df['ma20']) if params['ma_trend'] else True
    
    # 优化：竞价筛选（收紧区间）
    cond9 = daily_df['auction_rise_ratio'].between(
        params['high_open_min'], 
        params['high_open_max']
    )  # 高开1%-2%
    cond10 = daily_df['auction_volume_ratio'] >= params['auction_volume_ratio_min']  # 竞价量比≥8%
    cond11 = daily_df['is_high_open'] == True
    
    # 组合筛选（11个条件，排除伪加速）
    total_cond = cond1 & cond2 & cond3 & cond4 & cond5 & cond6 & cond7 & cond8 & cond9 & cond10 & cond11
    filtered_df = daily_df[total_cond].copy()
    
    log_msg(f"优化版形态筛选：符合条件{len(filtered_df)}只（目标100-200）")
    if len(filtered_df) < 80:
        log_msg(f"⚠️ 标的不足，建议放宽：单日涨幅≥0.3% 或 连续量能≥2天 或 竞价量比≥6%")
    elif len(filtered_df) > 220:
        log_msg(f"⚠️ 标的过多，建议收紧：单日涨幅≤4% 或 连续量能≥4天 或 竞价量比≥10%")
    return filtered_df

# --------------------------
# 步骤3：排序优化（新增量价配合得分）
# --------------------------
def score_by_short_term_factors(filtered_df):
    if len(filtered_df) == 0:
        return filtered_df
    df = filtered_df.copy()
    weights = CONFIG["sort_weights"]
    
    # 1. 竞价得分（优化：精准化计算）
    # 高开强度得分（1%-2%线性映射到0-50分）
    df['auction_rise_score'] = ((df['auction_rise_ratio'] - 0.01) / (0.02 - 0.01 + 1e-8)) * 50
    # 竞价量比得分（8%-20%线性映射到0-50分）
    df['auction_vol_score'] = ((df['auction_volume_ratio'] - 0.08) / (0.2 - 0.08 + 1e-8)) * 50
    df['auction_score'] = (df['auction_rise_score'] + df['auction_vol_score']).clip(0, 100)
    
    # 2. 新增：量价配合得分（量能+价格涨幅）
    # 量能得分（0.9-2.0线性映射）
    df['volume_strength_score'] = ((df['volume_ratio_5d'] - 0.9) / (2.0 - 0.9 + 1e-8)) * 50
    # 价格涨幅得分（0.5%-5%线性映射）
    df['price_strength_score'] = ((df['daily_rise_ratio'] - 0.005) / (0.05 - 0.005 + 1e-8)) * 50
    df['price_volume_score'] = (df['volume_strength_score'] + df['price_strength_score']).clip(0, 100)
    
    # 3. 趋势强度得分（连续上涨+均线）
    up_days_max = df['consecutive_up_days'].max() if df['consecutive_up_days'].max() > 0 else 1
    df['up_days_score'] = (df['consecutive_up_days'] / up_days_max) * 60
    df['ma_strength_score'] = ((df['ma5'] - df['ma20']) / df['ma20'].replace(0, 0.0001) * 1000).clip(0, 40)
    df['trend_strength_score'] = df['up_days_score'] + df['ma_strength_score']
    
    # 4. 风险过滤得分（RSI+布林带）
    df['rsi_risk_score'] = 100 - (abs(df['rsi14'] - 50) / 30) * 100
    df['bollinger_risk_score'] = (1.2 - df['close']/df['bollinger_lower']) / (0.2 + 1e-8) * 100
    df['risk_filter_score'] = (df['rsi_risk_score'] + df['bollinger_risk_score']).clip(0, 100) / 2
    
    # 综合得分
    df['total_score'] = (
        df['auction_score'] * weights['auction_score'] +
        df['price_volume_score'] * weights['price_volume_score'] +
        df['trend_strength_score'] * weights['trend_strength_score'] +
        df['risk_filter_score'] * weights['risk_filter_score']
    )
    
    # 取前150个
    df = df.sort_values(by='total_score', ascending=False).head(CONFIG["top_n"]).reset_index(drop=True)
    log_msg(f"优化版排序完成：最高得分{df['total_score'].max():.2f}，前50平均得分{df['total_score'].head(50).mean():.2f}")
    return df

# --------------------------
# 步骤4：单交易日处理（不变，保留优化后字段）
# --------------------------
def process_single_trade_date(df, trade_date):
    log_msg(f"\n===== 处理交易日：{trade_date.strftime('%Y-%m-%d')} =====")
    daily_df = df[df['date'].dt.date == trade_date].copy()
    if len(daily_df) < 1000:
        log_msg(f"⚠️ 当日数据异常，跳过")
        return pd.DataFrame()
    
    filtered_df = filter_single_shape(daily_df)
    if len(filtered_df) == 0:
        log_msg("⚠️ 无符合标的，跳过")
        return pd.DataFrame()
    
    ranked_df = score_by_short_term_factors(filtered_df)
    
    # 保留优化后关键字段（便于回测分组分析）
    result_cols = [
        'stock_code', 'date', 'close', 'open', 'volume',
        'consecutive_up_days', 'rise_ratio_30d', 'daily_rise_ratio',
        'volume_ratio_5d', 'consecutive_volume_days',
        'auction_rise_ratio', 'auction_volume_ratio', 'rsi14',
        'ma5', 'ma20', 'total_score', 'auction_score', 'price_volume_score'
    ]
    top_df = ranked_df[result_cols].reset_index(drop=True)
    top_df['trade_date'] = trade_date.strftime('%Y-%m-%d')
    
    date_str = trade_date.strftime('%Y%m%d')
    daily_save_path = os.path.join(CONFIG["daily_result_dir"], f"short_term_optimized_{date_str}.csv")
    top_df.to_csv(daily_save_path, index=False, encoding='utf-8-sig')
    log_msg(f"✅ 当日选股完成：{len(top_df)}只标的")
    return top_df

# --------------------------
# 主流程（不变）
# --------------------------
def run_short_term_selection():
    try:
        init_environment()
        df = pd.read_parquet(CONFIG["factortable_path"])
        df['date'] = pd.to_datetime(df['date'])
        df = validate_data(df)
        df = pre_calculate_indicators(df)
        
        trade_dates = sorted(df['date'].dt.date.unique())
        log_msg(f"检测到{len(trade_dates)}个交易日，开始优化版选股...")
        
        all_results = []
        for trade_date in trade_dates:
            daily_result = process_single_trade_date(df, trade_date)
            if not daily_result.empty:
                all_results.append(daily_result)
        
        if all_results:
            final_result = pd.concat(all_results, ignore_index=True)
            final_result.to_csv(CONFIG["selection_result_path"], index=False, encoding='utf-8-sig')
            log_msg(f"\n✅ 优化版选股完成！累计{len(final_result)}条记录，路径：{CONFIG['selection_result_path']}")
        else:
            log_msg(f"\n⚠️ 无选股结果，按日志提示放宽参数")
    except Exception as e:
        log_msg(f"❌ 选股失败：{str(e)}")
        raise

if __name__ == "__main__":
    run_short_term_selection()

[2025-10-24 12:56:15] ✅ 环境初始化完成，每日结果目录：./daily_short_term_optimized
[2025-10-24 12:56:16] ✅ 字段验证通过，共899630条记录
[2025-10-24 12:56:16] 开始计算优化版衍生指标...


C:\Users\jay\AppData\Local\Temp\ipykernel_34944\2275780062.py:81: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['rise_ratio_30d'] = df.groupby('stock_code', group_keys=False).apply(calc_30d_rise)
C:\Users\jay\AppData\Local\Temp\ipykernel_34944\2275780062.py:92: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('stock_code', group_keys=False).apply(calc_consecutive_volume)


[2025-10-24 12:56:32] ✅ 优化版衍生指标计算完成
[2025-10-24 12:56:32] 检测到175个交易日，开始优化版选股...
[2025-10-24 12:56:32] 
===== 处理交易日：2025-02-06 =====
[2025-10-24 12:56:32] 优化版形态筛选：符合条件0只（目标100-200）
[2025-10-24 12:56:32] ⚠️ 标的不足，建议放宽：单日涨幅≥0.3% 或 连续量能≥2天 或 竞价量比≥6%
[2025-10-24 12:56:32] ⚠️ 无符合标的，跳过
[2025-10-24 12:56:32] 
===== 处理交易日：2025-02-07 =====
[2025-10-24 12:56:33] 优化版形态筛选：符合条件0只（目标100-200）
[2025-10-24 12:56:33] ⚠️ 标的不足，建议放宽：单日涨幅≥0.3% 或 连续量能≥2天 或 竞价量比≥6%
[2025-10-24 12:56:33] ⚠️ 无符合标的，跳过
[2025-10-24 12:56:33] 
===== 处理交易日：2025-02-10 =====
[2025-10-24 12:56:33] 优化版形态筛选：符合条件0只（目标100-200）
[2025-10-24 12:56:33] ⚠️ 标的不足，建议放宽：单日涨幅≥0.3% 或 连续量能≥2天 或 竞价量比≥6%
[2025-10-24 12:56:33] ⚠️ 无符合标的，跳过
[2025-10-24 12:56:33] 
===== 处理交易日：2025-02-11 =====
[2025-10-24 12:56:33] 优化版形态筛选：符合条件0只（目标100-200）
[2025-10-24 12:56:33] ⚠️ 标的不足，建议放宽：单日涨幅≥0.3% 或 连续量能≥2天 或 竞价量比≥6%
[2025-10-24 12:56:33] ⚠️ 无符合标的，跳过
[2025-10-24 12:56:33] 
===== 处理交易日：2025-02-12 =====
[2025-10-24 12:56:33] 优化版形态筛选：符合条件0只（目标100-200）
[2025-10-24 12:56:33] ⚠️